In [1]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import seaborn as sns
sns.set_style("white")

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = io.loadmat("/media/zuziel/Windows/Users/Zuza/Documents/Programowanie/keras/notMNIST_small.mat")

In [4]:
X = data['images']
y = data['labels']
resolution = 28
classes = 10

X = np.transpose(X, (2, 0, 1))
X = X.reshape(X.shape[0], 1, 28, 28)

y = y.astype('int32')
X = X.astype('float32') / 255.

# 3 -> [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.]
Y = np_utils.to_categorical(y, 10)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

Sieć podobna do poprzedniej jednak ze zwiększonym rozmiarem warstwy - 512 neuronów zamiast 128.

In [8]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
from keras_ascii_sequential import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (1, 28, 28)
  Convolution2D    \|/  -------------------       832     0.0%
           relu   #####   (32, 24, 24)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (16, 12, 24)
        Dropout    | || -------------------         0     0.0%
                  #####   (16, 12, 24)
        Flatten   ||||| -------------------         0     0.0%
                  #####   (4608,)
          Dense   XXXXX -------------------   2359808    99.7%
           relu   #####   (512,)
          Dense   XXXXX -------------------      5130     0.2%
        softmax   #####   (10,)


In [10]:
# checkpoint
filepath="net2_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 32s - loss: 0.4304 - acc: 0.8756 - val_loss: 0.2807 - val_acc: 0.9207
Epoch 2/10
14979/14979 [==============================] - 31s - loss: 0.2235 - acc: 0.9344 - val_loss: 0.2692 - val_acc: 0.9196
Epoch 3/10
14979/14979 [==============================] - 32s - loss: 0.1577 - acc: 0.9517 - val_loss: 0.2445 - val_acc: 0.9343
Epoch 4/10
 8224/14979 [===============>..............] - ETA: 14s - loss: 0.0946 - acc: 0.9686 - ETA: 32s - loss: 0.1233 - acc: 0.9688

In [8]:
print(history.history)

{'val_loss': [0.28072433452580736, 0.26922110331870208, 0.24449055199231579, 0.23160877409621139, 0.2875405340362216, 0.25267016969233236, 0.31280956359826356, 0.33247191291028888, 0.33951722865697936, 0.34756987336870698], 'val_acc': [0.92069425901201607, 0.9196261682242991, 0.9343124165554072, 0.93511348464619493, 0.92843791722296398, 0.93724966622162886, 0.93297730307076099, 0.93484646194926568, 0.93110814419225629, 0.93377837116154871], 'acc': [0.87562587622671739, 0.93444155150942021, 0.95166566526470386, 0.96621937378997258, 0.97563255223980239, 0.98237532545563788, 0.98504573068963219, 0.98744909540022696, 0.991187662727819, 0.99359102743841377], 'loss': [0.43040788831868604, 0.22345956741084291, 0.15774777299410286, 0.10730094175675854, 0.076181516807527647, 0.057154015866334867, 0.048808199488949752, 0.041494993957332083, 0.027393859930441435, 0.023897857679288401]}


W tym wypadku już po pierwszych 10 iteracjach dochodzimy do wyników podobnych do tych widocznych po 20 iteracji.

In [9]:
score = model.evaluate(X,Y)
print(score)

18720/18724 [============================>.] - ETA: 0s    64/18724 [..............................] - ETA: 12s

In [10]:
print(model.evaluate(X_test,Y_test))

3744/3745 [============================>.] - ETA: 0s  64/3745 [..............................] - ETA: 2s

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 33s - loss: 0.0303 - acc: 0.9907 - val_loss: 0.3535 - val_acc: 0.9314
Epoch 2/10
14979/14979 [==============================] - 34s - loss: 0.0241 - acc: 0.9932 - val_loss: 0.3988 - val_acc: 0.9303
Epoch 3/10
14979/14979 [==============================] - 34s - loss: 0.0282 - acc: 0.9917 - val_loss: 0.3820 - val_acc: 0.9364
Epoch 4/10
14784/14979 [============================>.] - ETA: 0s - loss: 0.0193 - acc: 0.9945 

In [12]:
print(history.history)

{'val_loss': [0.35354591963268089, 0.39875552017037796, 0.38196449344940436, 0.39387634424750728, 0.41880491120474678, 0.40363210738115562, 0.40114104933539363, 0.43596750809699147, 0.52211623102406635, 0.44721752553152416], 'val_acc': [0.93137516688918554, 0.93030707610146868, 0.93644859813084114, 0.93698264352469962, 0.93164218958611478, 0.93190921228304402, 0.93164218958611478, 0.93030707610146868, 0.92122830440587455, 0.93538050734312417], 'acc': [0.99072034181186996, 0.99319046665331467, 0.99172174377461775, 0.99445890913946189, 0.99586087188730887, 0.99432538887776223, 0.99412510848521263, 0.99472594966286132, 0.99499299018626075, 0.9961279124107083], 'loss': [0.030330500978312705, 0.02408874792191085, 0.028201309312610234, 0.019394442220374074, 0.014386945564073628, 0.020077870763551962, 0.023207656097792882, 0.01839782451140955, 0.01515799974943793, 0.013612234830716394]}


Kolejne 10 iteracji niewiele zmieniło. Jeśli chodzi o zbiór testowy najwyższy wynik uzyskano w 6. iteracjiz pierwszego uczenia.

In [13]:
best_model =  Sequential()
best_model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Dropout(0.2))
best_model.add(Flatten())
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(classes, activation='softmax'))
# Compile model
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
best_model.load_weights("net2_weights-improvement-05-0.94.hdf5")

In [14]:
print(best_model.evaluate(X,Y))
print(best_model.evaluate(X_train,Y_train))
print(best_model.evaluate(X_test,Y_test))

3744/3745 [============================>.] - ETA: 0s

# Poszukiwanie najczęściej mylonych liter

In [15]:
predictions = best_model.predict(X)

In [16]:
errors = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
for i in range(0,X.shape[0]):
    errors[np.argmax(Y[i])][np.argmax(predictions[i])] += 1
print(errors)

[[1847, 6, 2, 4, 1, 0, 2, 8, 1, 1], [2, 1848, 4, 5, 7, 0, 4, 3, 0, 0], [0, 5, 1838, 1, 14, 1, 13, 0, 1, 0], [0, 13, 2, 1846, 0, 1, 3, 3, 1, 4], [1, 5, 9, 1, 1844, 3, 5, 1, 3, 1], [1, 5, 2, 5, 11, 1834, 3, 3, 4, 4], [3, 7, 22, 2, 6, 2, 1829, 0, 0, 1], [8, 7, 2, 1, 1, 0, 1, 1851, 1, 0], [6, 5, 3, 11, 5, 0, 3, 4, 1813, 22], [3, 3, 2, 13, 0, 1, 2, 4, 19, 1825]]


Wynika z tego, że najczęściej mylone są:
 1. G z C (22 razy)
 .. I z J (22 razy)
 3. J z I (19 razy)
 4. C z E (14 razy)
 6. C z G (13 razy)
 .. D z B (13 razy)
 .. J z D (13 razy)
 9. F z E (11 razy)
  . I z D (11 razy)

Wyniki te są lepsze niż w przypadku sieci II ale są nieco gorsze od sieci I